In [17]:
import pandas as pd
import requests
import PyPDF2
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

In [7]:
class ESGReport:
    def __init__(self,year,com_ticker):
        self.url = f'https://www.responsibilityreports.com/HostedData/ResponsibilityReports/PDF/NYSE_{comp_ticker}_{year}.pdf'
    
    def download_report(self):
        response = requests.get(self.url)
        with open(f'/tmp/company_{comp_ticker}.pdf', 'wb') as f:
            f.write(response.content)
        updated_file = self.delete_page()
        return updated_file
    
    def delete_page(self):
        file = f'/tmp/company_{comp_ticker}.pdf'
        file_reader = open(file, 'rb')
        file_pdf_reader = PyPDF2.PdfFileReader(file_reader)
        pdf_writer = PyPDF2.PdfFileWriter()
        pages_to_keep = list(range(27,60))
        for page_num in range(0, file_pdf_reader.numPages):
            if page_num in pages_to_keep:
                pdf_writer.addPage(file_pdf_reader.getPage(page_num))

        updated_file = file.replace('.pdf','_modified.pdf')
        output = open(updated_file, 'wb')
        pdf_writer.write(output)

        file_reader.close()
        output.close()        
        return updated_file


In [11]:
year = '2022'
comp_ticker = 'NEE'
endpoint = "https://formrecognizer-am.cognitiveservices.azure.com/"
cred = "5414fa99f51b45e4861cfcf1a6f749d7"

In [9]:
report = ESGReport(year,comp_ticker)

In [12]:
pdf_loc = report.download_report()

In [22]:
class ParseESGReport:
    def __init__(self,endpoint,keys,pdf_loc):
        self.endpoint = endpoint
        self.pdf_loc = pdf_loc
        self.keys = keys
    
    def fetchResult(self):
        form_recognizer_client = FormRecognizerClient(endpoint=self.endpoint, credential=AzureKeyCredential(self.keys))
        f = open(self.pdf_loc,'rb').read()
        result = form_recognizer_client.begin_recognize_content(f).result()
        return result
    
    def getupdateVal(self,ar):
        for idx in range(1,len(ar)):
            if ar[idx] - ar[idx-1] < 0.05:
                ar[idx] = ar[idx-1]
        return ar
    
    def smoothing(self,df,coordinate):
        temp = df.sort_values(coordinate)
        temp[f'{coordinate}_updated'] = self.getupdateVal(temp[coordinate].values.tolist())
        return temp
    
    def getPragraph(self):
        df = pd.DataFrame(columns=['pageNumber','Lines','x_min','y_min','x_max','y_max'])
        result = self.fetchResult()
        for res in result:
            pageNumber = res.page_number
            for line in res.lines:
                text = line.text
                x_min,y_min = list(line.bounding_box[0])
                x_max,y_max = list(line.bounding_box[2])
                df.loc[len(df)] = [pageNumber,text,x_min,y_min,x_max,y_max]
        df['word_len'] = df.Lines.apply(lambda x: len(x))
        page = df[(df.word_len>20)]
        page = self.smoothing(page,'x_min')
        page.sort_values(['x_min_updated','y_min'])
        paragraphs = []
        for name,grp in page.groupby(['pageNumber','x_min_updated']):
            paragraph = {}
            paragraph['page_number'] = name[0]
            paragraph['paragraph'] = ' '.join(grp.sort_values('y_min').Lines.values.tolist())
            paragraphs.append(paragraph)
        return paragraphs

In [23]:
parsing = ParseESGReport(endpoint,cred,pdf_loc)

In [24]:
paragraph = parsing.getPragraph()

In [25]:
paragraph

[{'page_number': 1,
  'paragraph': 'Vision and strategy: Our chairman, president and CEO NextEra Energy’s blueprint to reach Real Zero Confronting climate change NextEra Energy’s ESG journey Building the world’s leading clean energy provider Risks and opportunities 2021-2022 awards and recognitions'},
 {'page_number': 1,
  'paragraph': 'Managing and mitigating environmental risk Environmental risk identification, mitigation and management are key drivers for ensuring safe and sustainable operations. More than 150 corporate environmental professionals and numerous others embedded in our operating business units keep these environmental drivers top of mind, all of whom are key members of the teams that develop and operate our projects over the long term. These include experts in air emissions, water use and quality, remediation, wildlife and habitat, oil and hazardous substances, archaeology and cultural resources, and environmental policy, all critical to the responsible development and